In [ ]:
import os
import sys
import csv
import torch
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sys.path.append('../')
from datamodule.datamodule import select_data
from models.models import Classifier

In [ ]:
print(plt.style.available)
plt.style.use('seaborn-v0_8-dark-palette')

In [ ]:
path_data = '../../data/sim2real/'
files = os.listdir(path_data)
files.sort()
train_files = [os.path.join(path_data, i) for i in files if 'train' in i]
valid_files = [os.path.join(path_data, i) for i in files if 'valid' in i]